In [1]:
import sys
sys.path.append('/eos/user/s/sterbini/MD_ANALYSIS/public/')
#sys.path.append('/eos/project/l/liu/Toolbox/')
from myToolbox import *

from scipy.signal import savgol_filter
%matplotlib inline
plt.style.use('classic')

Version 0.03. This is the latest version.
Please help me to improve it reporting bugs to guido.sterbini@cern.ch.
Your platform is Linux-3.10.0-693.2.2.el7.x86_64-x86_64-with-redhat-6.9-Carbon
Your folder is /eos/user/f/fasvesta/2017
Your IP is 172.17.0.5
2017-12-20 16:05:49


In [2]:
import os
import glob
import pickle     
import numpy as np
import pandas as pnd
import platform 
import math
import sys
import time
from IPython.display import Image, display, HTML
from scipy.optimize import curve_fit

# function for tomoscope postprocessing
def extractProfile(fileName,NoB=1):
    '''Give me the input file (.dat) of the tomogram and I will give you 2 output:
        the deltaP_P and the the momentum distribution profile.
    ''' 

    print('Treating file ' + fileName)
    a=os.system('/eos/project/l/liu/Instrumentation/tomography/runofflinetomo ' + fileName)
    
    try:
        myFiles_c=glob.glob(fileName[0:-4] + '/d0*.data')
        myFiles_i=glob.glob(fileName[0:-4] + '/image0*.data')
        if NoB==1:
            convergence=np.loadtxt(myFiles_c[0])
            image=np.loadtxt(myFiles_i[0])
            with open(fileName[0:-4] + '/input_v2.dat','r') as stream: tomofileheader = stream.readlines()[:98]

            myInput={'timestamp': np.int(tomofileheader[1]),
                    'PLSUSER': tomofileheader[0][0:-1],
                    'NumberOfframes': np.int(tomofileheader[16][0:-1]),
                    'B_T': np.double(tomofileheader[75][0:-1]),
                    'Bdot_T_per_s': np.double(tomofileheader[77][0:-1]),
                    'machineRadius_m': np.double(tomofileheader[79][0:-1]),
                    'bendingRadius_m': np.double(tomofileheader[81][0:-1]),
                    'particleMass_eV': np.double(tomofileheader[85][0:-1]),
                    'h': np.int(np.double(tomofileheader[69][0:-1])),
                    'ctime': np.int(tomofileheader[2])
            }

            with open(fileName[0:-4] + '/plotinfo.data','r') as stream: tomofileheader = stream.readlines()

            myInput['profilecount']=np.int(str.split(tomofileheader[1])[2])
            myInput['profilelength']=np.int(str.split(tomofileheader[3])[2])

            myInput['dtbin']=np.double(str.split(tomofileheader[5])[2])
            myInput['dEbin']=np.double(str.split(tomofileheader[7])[2])
            myInput['xat0']=np.double(str.split(tomofileheader[11])[2])
            myInput['yat0']=np.double(str.split(tomofileheader[12])[2])
            myInput['eperimage']=np.double(str.split(tomofileheader[9])[2])

            mySpeedOfLight=299792458. #TODO
            myInput['momentum_eV']=myInput['B_T']*myInput['bendingRadius_m']/(10/mySpeedOfLight*1e8)*1.e9 #TODO
            myInput['totalEnergy_eV']=np.sqrt(myInput['momentum_eV']**2+myInput['particleMass_eV']**2)
            myInput['gamma']=myInput['totalEnergy_eV']/myInput['particleMass_eV']
            myInput['beta']=np.sqrt(1-1/myInput['gamma']**2)

            image=image*myInput['eperimage']/myInput['dtbin']/myInput['dEbin']

            halfProfileLength=myInput['profilelength']/2.

            Toffset= (myInput['xat0']-halfProfileLength)*myInput['dtbin']*1e9
            Eoffset= (myInput['yat0']-halfProfileLength)*myInput['dEbin']/1e6

            t=np.arange(-halfProfileLength,halfProfileLength)*myInput['dtbin']*1e9-Toffset
            E=np.arange(-halfProfileLength,halfProfileLength)*myInput['dEbin']/1e6-Eoffset

            deltaP_P= 1/myInput['beta']**2*E*1e6/myInput['totalEnergy_eV']
            myProfile=np.sum(np.reshape(image, [myInput['profilelength'], myInput['profilelength']]),0)
            myProfile=myProfile/np.trapz(myProfile,deltaP_P)
            myInput['deltaP_P']=deltaP_P;
            myInput['myProfile']=myProfile;
            myInput['E_MeV']=E
            myInput['t_ns']=t
            myInput['phaseSpace_e_per_eVs']=np.reshape(image, [myInput['profilelength'], myInput['profilelength']]);

            myProfileNormalized=myProfile/np.trapz(myProfile,deltaP_P)

            #rms emittance calculation
            xbar = 0.
            xms = 0.
            ybar = 0.
            yms = 0.
            xybar = 0.

            phaseSpaceNorm = myInput['phaseSpace_e_per_eVs'].T/np.sum(np.sum(myInput['phaseSpace_e_per_eVs'].T))

            for i in xrange(0,myInput['profilelength']):
                for j in xrange(0,myInput['profilelength']):
                    xbar += phaseSpaceNorm[i,j]*i
                    xms += phaseSpaceNorm[i,j]*i**2
                    ybar += phaseSpaceNorm[i,j]*j
                    yms += phaseSpaceNorm[i,j]*j**2
                    xybar += phaseSpaceNorm[i,j]*i*j

            myInput['rmsemittance'] = np.pi * myInput['dtbin'] * myInput['dEbin'] * np.sqrt((xms - xbar**2)*(yms - ybar**2) - (xybar -xbar*ybar)**2)    

            myMean=np.trapz(deltaP_P*myProfileNormalized,deltaP_P)   
            myRMS=np.sqrt(np.trapz((deltaP_P-myMean)**2*myProfileNormalized,deltaP_P))
            myInput['deltaP_P_RMS']=myRMS

            return myInput
    except:
        print ('Failed in '+ fileName)
        return np.nan

In [3]:
def fromMatlabToDataFrame(listing, listOfVariableToAdd, verbose=False, matlabFullInfo=False):
    listOfVariableToAdd=list(set(listOfVariableToAdd))
    myDataFrame=pnd.DataFrame({})
    cycleStamp=[]
    cycleStampHuman=[]
    PLS_matlab=[]
    matlabObject=[]
    matlabFilePath=[]
    for j in listOfVariableToAdd:
        exec(j.replace('.','_')+'=[]')
    for i in listing:
        if verbose:
            print(i)
        data=myToolbox.japcMatlabImport(i);
        if matlabFullInfo:
            matlabObject.append(data)
        #to correct
        localCycleStamp=np.nanmax(data.headerCycleStamps);
        deltaLocal_UTC=datetime.datetime.fromtimestamp(localCycleStamp/1e9)-datetime.datetime.utcfromtimestamp(localCycleStamp/1e9)
        utcCycleStamp=localCycleStamp+deltaLocal_UTC.total_seconds()*1e9
        cycleStamp.append(utcCycleStamp)
        aux=myToolbox.unixtime2datetimeVectorize(np.nanmax(data.headerCycleStamps)/1e9)
        cycleStampHuman.append(aux.tolist())
        PLS_matlab.append(data.cycleName)
        matlabFilePath.append(os.path.abspath(i))
        for j in listOfVariableToAdd:
            if hasattr(data,j.split('.')[0]):
                exec(j.replace('.','_') + '.append(data.' + j + ')')
            else:
                exec(j.replace('.','_') + '.append(np.nan)')
    myDataFrame['cycleStamp']=pnd.Series(cycleStamp,cycleStampHuman)
    myDataFrame['matlabPLS']=pnd.Series(PLS_matlab,cycleStampHuman)
    myDataFrame['matlabFilePath']=pnd.Series(matlabFilePath,cycleStampHuman)
    if matlabFullInfo:
        myDataFrame['matlabFullInfo']=pnd.Series(matlabObject,cycleStampHuman)
    for j in listOfVariableToAdd:
        exec('myDataFrame[\'' + j + '\']=pnd.Series(' +j.replace('.','_')+ ',cycleStampHuman)')    #myDataFrame=pnd.DataFrame({j:aux,
    return myDataFrame

In [4]:
def gaussian_5_parameters(x, c, m, A, mu, sig):
    """gaussian_5_parameter(x, c, m, A, mu, sig)"""
    return c+m*x+A/np.sqrt(2*np.pi)/sig*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

In [5]:
def makeGaussianFit_5_parameters(X,Y,for_df=False):
    try:      
        i = np.where( (X>min(X)+10) & (X<max(X)-10) )
        #i = np.where( (Y>1.01*min(Y)) & (Y<0.99*max(Y)) )
        X = X[i]
        Y = Y[i]
        indx_max = np.argmax(Y)
        mu0 = X[indx_max]
        window = 2*100
        x_tmp = X[indx_max-window:indx_max+window]
        y_tmp = Y[indx_max-window:indx_max+window]
        offs0 = min(y_tmp)
        ampl = max(y_tmp)-offs0
        x1 = x_tmp[np.searchsorted(y_tmp[:window], offs0+ampl/2)]
        x2 = x_tmp[np.searchsorted(-y_tmp[window:], -offs0+ampl/2)]
        FWHM = x2-x1
        sigma0 = np.abs(2*FWHM/2.355)
        ampl *= np.sqrt(2*np.pi)*sigma0
        slope = 0
        popt,pcov = curve_fit(gaussian_5_parameters,X,Y,p0=[offs0,slope,ampl,mu0,sigma0])
        if for_df:
            return popt[0], popt[3], popt[4]
        else:    
            return {'c': popt[0], 'm': popt[1],'A': popt[2], 'mu': popt[3], 'sig': popt[4], 'pcov': pcov, 'p': popt}
    except:
        if for_df:
            return np.nan,np.nan, np.nan
        else:
            return {k: np.nan for k in ['c', 'm','A', 'mu', 'sig', 'pcov', 'p']}

In [6]:
def PS_WSoptics(tunex, tuney, WS='64'):
    qv, qh, wire, betx, bety, dx, dy= np.loadtxt('/eos/user/f/fasvesta/optic_values.dat', dtype=([('qv',float),('qh',float),('wire',float),('betx',float),('bety',float),('dx',float),('dy',float)]),comments='#',unpack=True)
    startindx = {'54' :0, '64': 64, '65': 128, '68': 192, '85':256}
    stopindx = {'54' :64, '64': 128, '65': 192, '68': 256, '85':320}
    bx = betx[startindx[WS]:stopindx[WS]]
    by = bety[startindx[WS]:stopindx[WS]]
    Dx = dx[startindx[WS]:stopindx[WS]]
    qh1=qh[startindx[WS]:stopindx[WS]]
    qv1=qv[startindx[WS]:stopindx[WS]]
    betax = scipy.interpolate.interp2d(qh1,qv1,bx)
    betay = scipy.interpolate.interp2d(qh1,qv1,by)
    dispx = scipy.interpolate.interp2d(qh1,qv1,Dx)
    result ={'Beta_x':betax(tunex,tuney), 'Beta_y':betay(tunex,tuney),'D_x': dispx(tunex,tuney)}
    return result

In [7]:
def gaussian_5(x, c, m, A, mu, sig):
    """gaussian_5_parameter(x, c, m, A, mu, sig)"""
    return c+m*x+A*np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))
myDoubleGauss = lambda x,offs0,slope,mu,ampl1,sigma1,ampl2,sigma2: offs0+ x*slope + ampl1*np.exp(-(x-mu)**2/(2*sigma1**2)) + ampl2*np.exp(-(x-mu)**2/(2*sigma2**2))
def makeDoubleGaussianFit(X,Y):
    try:     
        i = np.where( (X>min(X)+10) & (X<max(X)-10) )
        X = X[i]
        Y = Y[i]
        indx_max = np.argmax(Y)
        mu0 = X[indx_max]
        window = 2*100
        x_tmp = X[indx_max-window:indx_max+window]
        y_tmp = Y[indx_max-window:indx_max+window]
        offs0 = min(y_tmp)
        ampl = max(y_tmp)-offs0
        x1 = x_tmp[np.searchsorted(y_tmp[:window], offs0+ampl/2)]
        x2 = x_tmp[np.searchsorted(-y_tmp[window:], -offs0+ampl/2)]
        FWHM = x2-x1
        sigma0 = np.abs(2*FWHM/2.355)
        ampl *= np.sqrt(2*np.pi)*sigma0
        slope = 0
        popt,pcov = curve_fit(myDoubleGauss,X,Y,p0=[offs0,slope,mu0,ampl,sigma0,0,max(X)])
        if abs(popt[4])<abs(popt[6]):
            A1 = popt[3]
            sig1 = abs(popt[4])
            A2 = popt[5]
            sig2 = abs(popt[6])
        else:
            A1 = popt[5]
            sig1 = abs(popt[6])
            A2 = popt[3]
            sig2 = abs(popt[4])
        return {'c': popt[0], 'm': popt[1], 'mu': popt[2], 'A1': A1, 'sig1': sig1, 'A2': A2, 'sig2': sig2, 'pcov': pcov, 'p': popt}
    except:
        return {k: np.nan for k in ['c', 'm', 'mu', 'A1', 'sig1', 'A2', 'sig2','pcov', 'p']}



In [8]:
def read_data(year=2017,month=7,date=18,hour=8,minutes=0,span=10,user='CPS%MD3',WS='85.V'):
    t1=datetime.datetime(year,month,date,hour,minutes)
    t2=t1+datetime.timedelta(hours=span)
    CALS=['PR.BWS.{}_ROT:PROF_DATA_IN'.format(WS),  'PR.BWS.{}_ROT:PROF_POSITION_IN'.format(WS),  'PR.BWS.{}_ROT:ACQ_DELAY'.format(WS)]
    df=myToolbox.fromTimberToDataFrame(CALS,t1,t2,fundamental=user)
    return  df

In [9]:
def read_psb_data(year=2017,month=21,date=11,hour=8,minutes=0,span=10,user='PSB%MD4',plane='V'):
    t1=datetime.datetime(year,month,date,hour,minutes)
    t2=t1+datetime.timedelta(hours=span)
    CALS=['BR3.BWS.2L1.'+plane+'_ROT:PROF_POSITION_IN', 'BR3.BWS.2L1.'+plane+'_ROT:PROF_DATA_IN']
    df=myToolbox.fromTimberToDataFrame(CALS,t1,t2,fundamental=user)
    return  df

In [10]:
def emittance_psb(ndf,plane='V',myFiles=None,dp=1.03e-3):
    betagamma=2.267
    bx=5.699
    by=4.25
    dx=1.466
    df=ndf.copy()
    a=df.columns
    if 'emittance '+plane not in a:
        df['emittance '+plane]=np.nan
    if 'sig '+plane not in a:
        df=profiles_psb(df,plane=plane)
    span=span_range(df,myFiles=myFiles)
    if plane=='V':
        df['emittance V'].iloc[span]=betagamma*(df['sig V'].iloc[span]**2)/by
    else:
        df['emittance H'].iloc[span]=betagamma*(df['sig H'].iloc[span]**2-(dx*1000)**2*dp**2)/bx
    return df

In [11]:
def emittance(ndf,WS='85.V',myFiles=None,beta=0.91444281513833,gamma=2.4708737618826,dp=1e-3,Qx=6.20,Qy=6.24):
    df=ndf.copy()
    a=df.columns
    if 'emittance' not in a:
        df['emittance']=np.nan
    if 'sig' not in a:
        df=profiles(df,WS=WS)
    span=span_range(df,myFiles=myFiles)
    if 'WP' not in a:
        df['WP']=np.nan
    df['WP'][span]='Qx={} Qy={}'.format(str(Qx),str(Qy))
    if WS[-1]=='V':
        df['emittance V'].iloc[span]=beta*gamma*(df['sig'].iloc[span]**2)/PS_WSoptics(Qx,Qy,WS=WS[:-2])['Beta_y']
    else:
        df['emittance H'].iloc[span]=beta*gamma*(df['sig'].iloc[span]**2-(PS_WSoptics(Qx,Qy,WS=WS[:-2])['D_x']*1000)**2*dp**2)/PS_WSoptics(Qx,Qy,WS=WS[:-2])['Beta_x']
    return df

In [ ]:
def twiss(ndf,WS='85.V',myFiles=None):
    df=ndf.copy()
    a=df.columns
    if 'betx' not in a:
        df['betx']=np.nan
    if 'bety' not in a:
         df['bety']=np.nan
    if 'dx' not in a:
         df['dx']=np.nan
    span=span_range(df,myFiles=myFiles)
    for i in span:
            df.loc[df.index[i],'betx']=PS_WSoptics(df['Qx'].iloc[i],df['Qy'].iloc[i],WS=WS[:-2])['Beta_x']
            df.loc[df.index[i],'dx']=PS_WSoptics(df['Qx'].iloc[i],df['Qy'].iloc[i],WS=WS[:-2])['D_x']
            df.loc[df.index[i],'bety']=PS_WSoptics(df['Qx'].iloc[i],df['Qy'].iloc[i],WS=WS[:-2])['Beta_y']
    return df

In [12]:
def emittance_df(ndf,WS='85.V',myFiles=None,beta=0.91444281513833,gamma=2.4708737618826,dp=1e-3):
    df=ndf.copy()
    a=df.columns
    if 'emittance V' not in a:
        df['emittance V']=np.nan
    if 'emittance H' not in a:
        df['emittance H']=np.nan
    if 'sig V' not in a:
        df=profiles(df,WS=WS)
    if 'sig H' not in a:
        df=profiles(df,WS=WS)
    span=span_range(df,myFiles=myFiles)
    if WS[-1]=='V':
        for i in span:
            df.loc[df.index[i],'emittance V']=beta*gamma*(df['sig V'].iloc[i]**2)/PS_WSoptics(df['Qx'].iloc[i],df['Qy'].iloc[i],WS=WS[:-2])['Beta_y']
    else:
        for i in span:
            df.loc[df.index[i],'emittance H']=beta*gamma*(df['sig H'].iloc[i]**2-(PS_WSoptics(df['Qx'].iloc[i],df['Qy'].iloc[i],WS=WS[:-2])['D_x']*1000)**2*dp**2)/PS_WSoptics(df['Qx'].iloc[i],df['Qy'].iloc[i],WS=WS[:-2])['Beta_x']
    return df

In [ ]:
def emittance_dp(ndf,WS='54.H',myFiles=None,beta=0.91444281513833,gamma=2.4708737618826):
    df=ndf.copy()
    a=df.columns
    if 'emittance dp H' not in a:
        df['emittance dp H']=np.nan
    if 'sig H' not in a:
        df=profiles(df,WS=WS)
    span=span_range(df,myFiles=myFiles)
    for i in span:
        df.loc[df.index[i],'emittance dp H']=beta*gamma*(df['sig H'].iloc[i]**2-(PS_WSoptics(df['Qx'].iloc[i],df['Qy'].iloc[i],WS=WS[:-2])['D_x']*1000)**2*df['deltaP_P_RMS'].iloc[i]**2)/PS_WSoptics(df['Qx'].iloc[i],df['Qy'].iloc[i],WS=WS[:-2])['Beta_x']
    return df

In [13]:
PS_WS_beta={'85.V':12.35,'64.V':22.67,'65.H':20.11,'68.H':13.26,'54.H':12.48}
PS_WS_app={'85.V':11.83,'64.V':22.04,'65.H':22.33,'68.H':12.62,'54.H':None}
PS_WS_D={'68.H':2.37,'65.H':3.2}

In [14]:
def intensity(myFiles,Qx=6.20,Qy=6.24):
    df=fromMatlabToDataFrame(myFiles,['PR_BCT_ST.Samples.value.samples'])
    def losses(a):
        try:
            b=(a[85]-a[1185])/a[85]*100 
            if b<0:
                print '85=',a[85],'1185=',a[1185]
        except:
            b=np.nan
        return b
    def ratio(a):
        try:
            c=a[1185]/a[85]
        except:
            c=np.nan
        return c
    df['WP']='Qx={} Qy={}'.format(str(Qx),str(Qy))
    df['losses']=df['PR_BCT_ST.Samples.value.samples'].apply(losses)
    df['ratio']=df['PR_BCT_ST.Samples.value.samples'].apply(ratio)
    return df
def intensity_df(ndf):
    df=ndf.copy()
    def losses(a):
        try:
            b=(a[85]-a[1085])/a[85]*100 
        except:
            b=np.nan
        return b
    def ratio(a):
        try:
            c=a[1085]/a[85]
        except:
            c=np.nan
        return c
    df['losses']=df['PR_BCT_ST.Samples.value.samples'].apply(losses)
    df['ratio']=df['PR_BCT_ST.Samples.value.samples'].apply(ratio)
    return df

In [15]:
def profiles_psb(ndf,plane='V',myFiles=None):
    df=ndf.copy()
    x=np.linspace(-40,40,1000)
    a=df.columns
    if 'profile '+plane not in a:
        df['profile '+plane]=df['BR3.BWS.2L1.'+plane+'_ROT:PROF_DATA_IN'].apply(filtered)
        df['position '+plane]=df['BR3.BWS.2L1.'+plane+'_ROT:PROF_POSITION_IN'].values/1000.        
    if 'sig '+plane not in a:
        df['sig '+plane]=np.nan
        df['mu '+plane]=np.nan
        df['c '+plane]=np.nan
    span=span_range(df,myFiles=myFiles)
    if np.isnan(df['sig '+plane].iloc[span]).all():
        for i in span:
            df['c '+plane].iloc[i],df['mu '+plane].iloc[i],df['sig '+plane].iloc[i]=makeGaussianFit_5_parameters(df['position '+plane].iloc[i],df['profile '+plane].iloc[i],for_df=True)
            try:
                yz=scipy.interpolate.interp1d(df['position '+plane].iloc[i]-df['mu '+plane].iloc[i],df['profile '+plane].iloc[i]-df['c '+plane].iloc[i])(x)
                if np.isnan(yz[0]):
                    yz=np.nan
            except:
                yz = np.nan
            df['profile '+plane].iloc[i]=yz
            df['position '+plane].iloc[i]=x
    return df

In [16]:
def filtered(a):
    b=(scipy.signal.savgol_filter(a,15,1))
    return b
def profiles(ndf,WS='85.V',myFiles=None):
    df=ndf.copy()
    x=np.linspace(-40,40,1000)
    a=df.columns
    if 'profile '+WS[-1] not in a:
        df['profile '+WS[-1]]=df['PR.BWS.{}_ROT:PROF_DATA_IN'.format(WS)].apply(filtered)
        df['position '+WS[-1]]=df['PR.BWS.{}_ROT:PROF_POSITION_IN'.format(WS)].values/1000.
    if 'sig '+WS[-1] not in a:
        df['sig '+WS[-1]]=np.nan
        df['mu '+WS[-1]]=np.nan
        df['c '+WS[-1]]=np.nan
    span=span_range(df,myFiles=myFiles)
    if np.isnan(df['sig '+WS[-1]].iloc[span]).all():
        for i in span:
            df['c '+WS[-1]].iloc[i],df['mu '+WS[-1]].iloc[i],df['sig '+WS[-1]].iloc[i]=makeGaussianFit_5_parameters(df['position '+WS[-1]].iloc[i],df['profile '+WS[-1]].iloc[i],for_df=True)
            try:
                yz=scipy.interpolate.interp1d(df['position '+WS[-1]].iloc[i]-df['mu '+WS[-1]].iloc[i],df['profile '+WS[-1]].iloc[i]-df['c '+WS[-1]].iloc[i])(x)
                if np.isnan(yz[0]):
                    yz=np.nan
            except:
                yz = np.nan
            df['profile '+WS[-1]].iloc[i]=yz
            df['position '+WS[-1]].iloc[i]=x
    return df

In [17]:
def span_range(df,myFiles=None):
    if myFiles:
        span=[i for i in xrange(len(df)) if datetime.datetime.strptime(myFiles[0][-27:-4], '%Y.%m.%d.%H.%M.%S.%f')<df.index[i]<datetime.datetime.strptime(myFiles[-1][-27:-4], '%Y.%m.%d.%H.%M.%S.%f')]
    else:
        span=xrange(len(df))
    return span

In [18]:
def second_moment(ndf,WS='85.V',myFiles=None,n_sigmas=4):
    df=ndf.copy()
    a=df.columns
    plane = WS[-1]
    fig=plt.figure()
    #if plane == 'H':
    if 'second_moment '+plane not in a:
        df['second_moment '+plane]=np.nan
    if 'profile '+plane not in a:
        df=profiles(df,WS=WS,myFiles=myFiles)
    span=span_range(df,myFiles=myFiles)
    for i in span:
        try:
            a=np.array(zip(df.loc[df.index[i],'position '+plane],df.loc[df.index[i],'profile '+plane]))     
            c=[j for j in np.where(a[:,0]<(n_sigmas+0.5)*df['sig '+plane].iloc[i])[0] if j in np.where(a[:,0]>n_sigmas*df['sig '+plane].iloc[i])[0]]
            d=np.mean(a[:,1][c])
            c=[j for j in np.where(a[:,0]>-(n_sigmas+0.5)*df['sig '+plane].iloc[i])[0] if j in np.where(a[:,0]<-n_sigmas*df['sig '+plane].iloc[i])[0]]
            d1=np.mean(a[:,1][c])
            b=[j for j in np.where(a[:,0]<n_sigmas*df['sig '+plane].iloc[i])[0] if j in np.where(a[:,0]>-n_sigmas*df['sig '+plane].iloc[i])[0]]
            a[:,1]-=np.nanmean([d,d1])
            plt.plot(a[:,0][b],a[:,1][b])
            W=sum(a[:,1][b])
            m=a[:,0][b]*a[:,1][b]
            M=sum(m)/W
            s=a[:,1][b]*(a[:,0][b]-M)**2
            S=np.sqrt(sum(s)/W)
            df.loc[df.index[i],'second_moment '+plane]=S  
            #print sum(s),W
        except:
            df.loc[df.index[i],'second_moment '+plane]=np.nan
    #plt.xlim(-4*dfh['sig'].mean(),4*dfh['sig'].mean())
    #plt.ylim(-0.1,0.7)
    return df ,fig